In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [3]:
# Loading camera calibration
cameraCalibration = pickle.load( open('./camera_cal/cam_calib_pickle.p', 'rb' ) )
mtx, dist = map(cameraCalibration.get, ('mtx', 'dist'))

# Load perspective transformed images
persptransform_images=pickle.load(open('./camera_cal/persptransform_images.p','rb'))
print(np.array(persptransform_images).shape)

# Load transformation matrix
transMatrix = pickle.load( open('./camera_cal/perspective_transform.p', 'rb' ) )
M, Minv = map(transMatrix.get, ('M', 'Minv'))

(8, 720, 1280, 3)


In [4]:
#Load test images
test_image_files = glob.glob('./test_images/*.jpg')
print("{} Test images loaded".format(len(test_image_files)))

test_images=[]

for f in test_image_files:
    t=cv2.imread(f)
    t=cv2.cvtColor(np.array(t), cv2.COLOR_BGR2RGB)
    test_images.append(t)

8 Test images loaded


In [5]:
def showImages(images, cols = 4, rows = 5, figsize=(15,10), cmap=None):
    imgLength = len(images)
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    indexes = range(cols * rows)
    for ax,indx in zip(axes.flat,indexes):
            image = images[indx]
            if cmap == None:
                ax.imshow(image)
            else:
                ax.imshow(image, cmap=cmap)
            ax.set_title(test_image_files[indx])
#showImages(persptransform_images, 2, 4, (15, 13))
#showImages(test_images, 2, 4, (15, 13))

In [ ]:
# Image processing pipeline
# Based on the images above, the S channel (index 2) returns the lanes more reliably
# Undistort and change color space to HLS
from collections import deque

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

leftx_base_runningavg=0
rightx_base_runningavg=0
lanebuffer_limit=7
lanecount=0
leftx_lanebasebuffer=deque([],maxlen=lanebuffer_limit)
rightx_lanebasebuffer=deque([],maxlen=lanebuffer_limit)

leftx_top_runningavg=0
rightx_top_runningavg=0
leftx_lanetopbuffer=deque([],maxlen=lanebuffer_limit)
rightx_lanetopbuffer=deque([],maxlen=lanebuffer_limit)

# This applies any threshold to an image
def thresholdimg(img, minthresh,maxthresh):
    img_binary=np.zeros_like(img)
    img_binary[(img>=minthresh) &(img<=maxthresh)]=1
    return img_binary

def sobelimg(img, xory='x', sobel_kernel=3,minthresh=0, maxthresh=255):
    if xory=='x':
        ord1=0
        ord2=1
    else:
        ord1=1
        ord2=0
    
    sobel=cv2.Sobel(img, cv2.CV_64F, ord2, ord1, ksize=sobel_kernel)
    abs_sobel=np.absolute(sobel)
    scaled=np.uint8(255.0*abs_sobel/np.max(abs_sobel))
    return thresholdimg(scaled,minthresh, maxthresh)

def gradients(img,minthresh=0, maxthresh=255):
    #Process incomming image: undistort->convert to HLS->isolate s and l channel->
    #combine s and l channel-> Sobel along x and y -> combine
    undist=cv2.undistort(img,mtx,dist,None,mtx)
    img=cv2.cvtColor(undist,cv2.COLOR_RGB2HLS)
    imgs=img[:,:,1]
    imgl=img[:,:,2]
    imgsl=imgs+imgl
    sobelX = sobelimg(imgsl, 'x',3,minthresh,maxthresh)
    sobelY = sobelimg(imgsl, 'y',3,minthresh,maxthresh)
    combined = np.zeros_like(sobelX)
    combined [(sobelX == 1) & (sobelY ==1)] = 1
    return combined


def region_of_interest(img):
    
    shape = img.shape
    vertices = np.array([[(0,20),(shape[1],20),(6*shape[1]/6.8,shape[0]),
                      (shape[1]/10,shape[0]), (0,20)]],dtype=np.int32)

    mask = np.zeros_like(img)   
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def determinelanes(binary_warped):
    global leftx_base_runningavg
    global rightx_base_runningavg
    global lanecount
    global lanebuffer_limit
    global leftx_lanebasebuffer
    global rightx_lanebasebuffer
    
    global leftx_top_runningavg
    global rightx_top_runningavg
    global leftx_lanetopbuffer
    global rightx_lanetopbuffer
    
    #binary_warped=colorandpersp_images[2]
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint]) #argmax returns indices of maximum values
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
        
    if leftx_base_runningavg==0 and lanecount==0:
        leftx_base_runningavg=leftx_base
        rightx_base_runningavg=rightx_base
        leftx_lanebasebuffer.append(leftx_base_runningavg)
        rightx_lanebasebuffer.append(rightx_base_runningavg)
        
    if lanecount<=lanebuffer_limit:
        leftx_base_runningavg=(leftx_base+leftx_base_runningavg)/2
        rightx_base_runningavg=(rightx_base+rightx_base_runningavg)/2
        leftx_lanebasebuffer.append(leftx_base)
        rightx_lanebasebuffer.append(rightx_base)
        
    if lanecount>lanebuffer_limit:
        if np.abs(rightx_base_runningavg-rightx_base)>50:
            rightx_base=rightx_base_runningavg
        if np.abs(leftx_base_runningavg-leftx_base)>30:
            leftx_base=leftx_base_runningavg
            
        leftx_lanebasebuffer.popleft()
        rightx_lanebasebuffer.popleft()
        leftx_lanebasebuffer.append(leftx_base)
        rightx_lanebasebuffer.append(rightx_base)
        leftx_base_runningavg=np.average(leftx_lanebasebuffer)
        rightx_base_runningavg=np.average(rightx_lanebasebuffer)
        
    #print(lanecount, rightx_base)    
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero() #nonzero returns indices of nonzero values
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = np.int(leftx_base_runningavg)
    rightx_current = np.int(rightx_base_runningavg)
    # Set the width of the windows +/- margin
    margin = 110
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
   
        
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
  
    #if lanecount in [46,69,119,129,167,170,173,175]:
    #    print(lanecount+1, right_fitx,left_fitx)
    # Calculate the new radii of curvature
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    return out_img, left_fitx, right_fitx, ploty, left_curverad, right_curverad, left_fit, right_fit, left_fit_cr, right_fit_cr

def unwarpimg(warped,test_image,left_fitx,right_fitx, img_size, left_curverad, right_curverad, left_fit_cr, right_fit_cr):
    ploty = np.linspace(0, 719, num=720)
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (test_image.shape[1], test_image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(test_image, 1, newwarp, 0.3, 0)
    
    #calculate vehicle center
    xMax = test_image.shape[1]*xm_per_pix
    yMax = test_image.shape[0]*ym_per_pix
    vehicleCenter = xMax / 2
    lineLeft = left_fit_cr[0]*yMax**2 + left_fit_cr[1]*yMax + left_fit_cr[2]
    lineRight = right_fit_cr[0]*yMax**2 + right_fit_cr[1]*yMax + right_fit_cr[2]
    lineMiddle = lineLeft + (lineRight - lineLeft)/2
    diffFromVehicle = lineMiddle - vehicleCenter
    if diffFromVehicle > 0:
        message = '{:.2f} m right'.format(diffFromVehicle)
    else:
        message = '{:.2f} m left'.format(-diffFromVehicle)
    
    # Write on image
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontColor = (0,0, 255)
    rad_of_curve = 0.5*(round(right_curverad,1) + round(left_curverad,1))
    cv2.putText(result, 'Radius of curvature: {:.0f} m'.format(rad_of_curve), (430, 630), font, 1, fontColor, 2 ,cv2.LINE_AA)
    cv2.putText(result, 'Vehicle is {} of center'.format(message), (430, 670), font, 1, fontColor, 2 ,cv2.LINE_AA)
    return result

In [ ]:
from moviepy.editor import *
import imageio

def videopipeline(test_image):
    global lanecount
    '''
    #[166,167,168,169,170,171,172,173]
    if lanecount in [46,69,119,129,167,170,173,175]:
        fname = 'C:/Users/omkar.karve/CarND-Advanced-Lane-Lines-master/output_images/'+str(lanecount+1)+"_orig.jpg"
        temp=cv2.cvtColor(test_image,cv2.COLOR_BGR2RGB)
        cv2.imwrite(fname,temp)
    '''
    img=gradients(test_image ,10,160)
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size)
    warped = region_of_interest(warped)
    out_img, left_fitx, right_fitx, ploty, left_curverad, right_curverad, \
    left_fit, right_fit, left_fit_cr, right_fit_cr = determinelanes(warped)
    result = unwarpimg(warped,test_image,left_fitx,right_fitx, img_size, left_curverad, right_curverad, left_fit_cr, right_fit_cr)
    lanecount=lanecount+1
    '''
    #[167,168,169,170,171,172,173,174]
    if lanecount in [47,70,120,130,168,171,174,175]:
        fname = 'C:/Users/omkar.karve/CarND-Advanced-Lane-Lines-master/output_images/'+str(lanecount)+"_conv.jpg"
        temp=cv2.cvtColor(result,cv2.COLOR_BGR2RGB)
        cv2.imwrite(fname,temp)
    '''
    return result

in_clip = VideoFileClip("20180406144644a-noaudio.mp4")
out_clip = in_clip.fl_image(videopipeline)
%time out_clip.write_videofile('./20180406144644a-noaudio_processed.mp4', audio=False,progress_bar=False)

[MoviePy] >>>> Building video ./20180406144644a-noaudio_processed.mp4
[MoviePy] Writing video ./20180406144644a-noaudio_processed.mp4
